# 概要
## リサーチクエスチョン
- センサーの多様化
- 取得情報精度向上（ユーザ状態の正確な把握）
## 結論
- 取得情報精度向上は高評価企業の方が出願比率が高い。→○
- センサーの多様化は△。高評価企業の方で心拍センサや、モーションセンサ（角速度等）で差が見られたが、多様化しているという結論は下せない。
## その他の項目（高評価企業の方が出願比率が高い）
- スポーツ系の用途で有意な差
- データ解析、集計、シュミレーションで有意な差
## 総合
- 高評価企業の特徴を捉えやすい結果となった
- 以下、推測。高評価企業はスポーツ用途に注力している。その、取り組みの内容として、心拍センサ、各種モーションセンサで活動を計測する情報種の幅を広げ、またその分析についても注力している。スポーツユーザーでは、成果の可視化（スピード、消費エネルギー、フォーム等）に関心があり、これに対応するための開発であることが伺える。
- 健康管理などソフトな使い方をするユーザーにとって、上記のような機能の充実化は魅力にはならない。そうなると、ウェアラブル先行企業は価格競争において勝負にならないため、高付加価値で勝負したい企業はスポーツ用途の市場にシフトしていく可能性がある。

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

/home/kyohei/anaconda2/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


In [2]:
data_amazon = pd.read_csv('api_data_listbands.csv')

# 企業分類
- レビュー件数が10件以上の製品を製造している企業に限定する。
- 高評価企業はレビュー値が4.07以上(実質4.1)
- 中評価企業はレビュー値が4.06〜3.0（実質4.0〜3.0）
- 高評価企業のうち出願が見られたのは、Xiaomi、Fitbit、MISFIT WEARABLE、GARMIN(ガーミン)、HUAWEI、Wahoo Fitness、POLAR(ポラール)
- 中評価企業のうち出願が見られたのは、Jawbone、Sony

# 特許データ整理


In [3]:
high_company = [u'フィットビット',u'GARMIN SWITZERLAND GMBH',u'POLAR ELECTRO OY']
midle_company = [u'Aliphcom',u'ソニー']
other_company = [u'SILICON VALLEY BANK AS ADMINISTRATIVE AGENT',u'BLACKROCK ADVISORS LLC']

In [4]:
data_pat = pd.read_csv('/home/kyohei/wearable/data/01.文献リスト_特許.csv',encoding='SHIFT-JIS')
categories = pd.read_csv('/home/kyohei/wearable/data/T_小区分.csv',encoding='SHIFT-JIS')
high_com_pat_all = data_pat[data_pat[u'筆頭出願人名'].isin(high_company)]
mid_com_pat_all = data_pat[data_pat[u'筆頭出願人名'].isin(midle_company)]

/home/kyohei/anaconda2/lib/python2.7/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
high_com_pat_list = high_com_pat_all[high_com_pat_all[u'B0301']==1]
mid_com_pat_list = mid_com_pat_all[mid_com_pat_all[u'B0301']==1]

In [6]:
print('高評価企業の内訳\n%s')% high_com_pat_list[u'筆頭出願人名'].value_counts()
print('中評価企業の内訳\n%s')% mid_com_pat_list[u'筆頭出願人名'].value_counts()

高評価企業の内訳
フィットビット                    47
POLAR ELECTRO OY            5
GARMIN SWITZERLAND GMBH     3
Name: 筆頭出願人名, dtype: int64
中評価企業の内訳
Aliphcom    44
ソニー         30
Name: 筆頭出願人名, dtype: int64


In [7]:
#企業の限定
high_analysis = high_com_pat_list[high_com_pat_list[u'筆頭出願人名'] == u'フィットビット']
mid_analysis = mid_com_pat_list

In [8]:
#各技術区分ごとに比率を計算する
high_analysis_cal = high_analysis.iloc[:,19:].count() / float(high_analysis.shape[0])
mid_analysis_cal = mid_analysis.iloc[:,19:].count() / float(mid_analysis.shape[0])
compare_category = pd.concat([high_analysis_cal,mid_analysis_cal],axis=1)
compare_category.rename(columns={0:"high",1:"mid"},inplace=True)
compare_category["diff"] = compare_category["high"] - compare_category["mid"]
compare_category["diff_abs"] = abs(compare_category["high"] - compare_category["mid"])

categories = categories.set_index(u"記号",drop=True)
compare_category = pd.concat([categories,compare_category],axis=1)

In [9]:
#高評価、中評価の値代入
high_analysis['high'] = 1
mid_analysis['high'] = 0
#高評価データと中評価データを合体
data_all = pd.concat([high_analysis,mid_analysis])
#NaNを代入
data_all = data_all.fillna(0)
#技術区分の列に限定
data_all_cal = data_all.iloc[:,19:]
#何もついていない項目、全部埋まっている項目は評価から除く
data_all_cal = data_all_cal.loc[:,((data_all_cal.sum() > 0) & (data_all_cal.sum() < data_all_cal.shape[0]))]

from scipy.stats import chi2_contingency
#評価する技術区分を取得
columns = data_all_cal.columns[:-1]
pvalues = []
for row in columns:
    #2（高・中）×2（有・無）のテーブル
    table = pd.crosstab(data_all_cal['high'],data_all_cal[row])
    #χ二乗検定
    chisq_value,pvalue,df,expected = chi2_contingency(table,correction=False) #補正無し:correction=False
    pvalues.append([row,pvalue,chisq_value])

pvalues_df = pd.DataFrame(np.array(pvalues)[:,1],index=np.array(pvalues)[:,0],columns=['pvalue'])
compare_category = pd.concat([compare_category,pvalues_df],axis=1)
compare_category['pvalue'] = compare_category['pvalue'].astype(float)

/home/kyohei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/home/kyohei/anaconda2/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


# リサーチクエスチョン
- センサーの多様化
- 取得情報精度向上（ユーザ状態の正確な把握）

# 分析結果
# プラス域

In [13]:
compare_category[(compare_category['pvalue']<0.05) & (compare_category['diff']>0.0)].sort_values('diff',ascending=False).style.bar(subset=['succeed','normal','diff_abs'],color='#d65f5f')

# マイナス域

In [16]:
compare_category[(compare_category['pvalue']<0.05) & (compare_category['diff']<0.0)].sort_values('diff_abs',ascending=False).style.bar(subset=['succeed','normal','diff_abs'],color='#d65f5f')

# 分析結果（中区分ごと）

### 効果・価値
- ユーザ状態の正確な把握は高評価企業が高い値。リサーチクエスチョン採択。
- 中評価企業は安全性向上と作業効率向上が高い。業務用途のはなしか。
- コミュニケーションの円滑化？？？

In [18]:
compare_category[compare_category.index.str.startswith("C01")].style.bar(subset=['succeed','normal','diff_abs'],color='#d65f5f')

### 全体設計デザイン
- 目立った差はない。リストバンドについて、このあたりはひと通り開発は終わった？

In [19]:
compare_category[compare_category.index.str.startswith("D01")].style.bar(subset=['succeed','normal','diff_abs'],color='#d65f5f')

### モーションセンサー
- 振動、角速度、ジャイロセンサーで高評価企業が高い
- 圧力、ひずみは差はなし

In [20]:
compare_category[compare_category.index.str.startswith("F01")].style.bar(subset=['succeed','normal','diff_abs'],color='#d65f5f')

### バイタルセンサー
- 心拍センサは出願比率が高い。高評価企業が高い。次に来るセンサの本命に注力か
- 体温、発汗は中評価企業の出願比率が高い

In [22]:
compare_category[compare_category.index.str.startswith("F02")].style.bar(subset=['succeed','normal','diff_abs'],color='#d65f5f')

### 5.5 医療用途
- 医療用途は中評価企業で多いが、比率自体は低い。権利を広く取りたいからの可能性が高いのでフォーカスしない。

In [24]:
compare_category[compare_category.index.str.startswith("H02")].style.bar(subset=['succeed','normal','diff_abs'],color='#d65f5f')

### 5.6 ヘルスケア用途
- 健康、生活記録の比率が両者ともに高く。有意な差はない。当たり前といえばあたりまえ

In [26]:
compare_category[compare_category.index.str.startswith("H03")].style.bar(subset=['succeed','normal','diff_abs'],color='#d65f5f')

### 5.7 スポーツ用途
- 全体的に高評価企業のほうが出願比率が高い。
- 健康を管理するよりも、成果を可視化するほうが消費者にとって価値があるのか。
- 一般の消費者よりも、スポーツ愛好家のほうがウェアラブルが魅力的に映る。

In [27]:
compare_category[compare_category.index.str.startswith("H04")].style.bar(subset=['succeed','normal','diff_abs'],color='#d65f5f')

### 5.8 バイタルデータ
- センサでは、心拍は高評価、脈拍は中評価という対応だったが、それが反映されていないので、取りこぼしがある。
- データに関しては、深く突っ込まないほうが無難

In [28]:
compare_category[compare_category.index.str.startswith("I07")].style.bar(subset=['succeed','normal','diff_abs'],color='#d65f5f')